In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables.history import RunnableWithMessageHistory
import streamlit as st
from langchain_chroma import Chroma
from langchain_core.prompts import MessagesPlaceholder

import os 
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
# groqapi_key=os.getenv("GROQ_API_KEY")
# llm=ChatGroq(model="",api_key=groqapi_key)
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
st.title("Conversational RAG With PDF uploads and caht History")
st.write("Uplaod the PDF and chat with it ")
api_key=st.text_input("enter your groq api key")

if api_key:
    llm=ChatGroq(api_key=api_key,model="Gemma2-9b-It")
    sesion_id=st.text_input("Session ID",value="default_session")
    if "store" not in st.session_state:
        st.session_state.store={}
    uploadedfiles=st.file_uploader("Choose a pdf files",type="pdf",accept_multiple_files=True)
    if uploadedfiles:
        documents=[]
        for uploadedfile in uploadedfiles:
            temppdf=f"./temp.pdf"
            with open(temppdf,"wb") as file:
                file.write(uploadedfile.getvalue())
                file.name=uploadedfile.name
            loader=PyPDFLoader(temppdf)
            docs=loader.load()
            documents.extend(docs)
        textsplitter=RecursiveCharacterTextSplitter(chunk_size=5000,chunk_overlap=200)
        splits=textsplitter.split_documents(documents)
        vectorstore=Chroma.from_documents(splits,embedding=embeddings)
        retriever=vectorstore.as_retriever()

        contextualize_q_system_prompt=(
        "Given a chat history and the latest user question"
        "which might reference context in the chat history, "
        "formulate a standalone question which can be understood "
        "without the chat history. Do NOT answer the question, "
        "just reformulate it if needed and otherwise return it as is."
        )

        contextualize_q_prompt= ChatPromptTemplate.from_messages(

        [

        ("system",contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}")
        ]


        )

        history_aware_retriver=create_history_aware_retriever(llm,retriever,contextualize_q_prompt)
        system_prompt = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the "
        "answer concise."
        "\n\n"
        "{context}"
        )
        qa_prompt = ChatPromptTemplate.from_messages(
        [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
        ]
        )
        question_answer_chain=create_stuff_documents_chain(llm,qa_prompt)
        rag_chian=create_retrieval_chain(history_aware_retriver,question_answer_chain)

        def get_session_history(session_id:str)->BaseChatMessageHistory:

            if sesion_id not in st.session_state.store:
             
             st.session_state.store[sesion_id]=ChatMessageHistory()
             return st.session_state.store[sesion_id]
             

        

        consersational_rag_chain=RunnableWithMessageHistory(

             rag_chian,
            input_messages_key="input",
            history_messages_key="chat_history",
            output_messages_key="answer"


            )

        user_input=st.text_input("your question")
        if  user_input:
            session_history=get_session_history(sesion_id)

            response=consersational_rag_chain.invoke(
                {"input":user_input},
                config={
                    "configurable":{"session_id":sesion_id}
                },

            )
            st.write(st.session_state.store)
            st.write("Assistant",response['answer'])
            st.write("chat history",session_history.messages)

# RAG Q&A Conversation With PDF Including Chat History